In [1]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(group_id, network_dataset, locations, fc_SA):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source= network_dataset,
        layer_name="{}_SA_400m".format(group_id),
        travel_mode="Walk",
        travel_direction="FROM_FACILITIES",
        cutoffs=[400],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS",
        polygon_detail="HIGH",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="10 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )
    arcpy.na.AddLocations(
        in_network_analysis_layer=salyr,
        sub_layer="Facilities",        
        in_table=locations,
        field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Meters # 0;Attr_pedestrian_minutes # 0;Breaks_Minutes # #;Breaks_Meters # #;Breaks_pedestrian_minutes # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="nw SHAPE;Network_ND_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="NO_SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )
    print("Solving network analysis")
    arcpy.na.Solve(
        in_network_analysis_layer=salyr,
        ignore_invalids="SKIP",
        terminate_on_solve_error="CONTINUE",
        simplification_tolerance=None,
        overrides=""
    )
    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
    arcpy.Delete_management(salyr) 

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'step1_GQA')
step2_f = os.path.join(indata_f, 'OutputData', 'step2_GQA_Final')
step3_f = os.path.join(indata_f, 'OutputData', 'step3_SA')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_Input_MultiUA.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"

    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for ctry
ctry = 'FI'
sql_query = "Batch >0 "

# Define the path and name for the geodatabase
gdb_path = os.path.join(indata_f, 'Processing')

gdb_name = f"SA_output_1911_{ctry}.gdb"
outGDB = os.path.join(gdb_path, gdb_name)
# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
        inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
        fc_SA = os.path.join(outPath, "{}_SA.shp".format(uc_code))
        ####fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
        if ctr_cd.startswith("CH-"):
            ctr_cd = "CH"

        ##if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO" and ctr_cd!="CH":
        if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd==ctry:
            print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
            pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
            if not arcpy.Exists(pedestrianNWpath):
                raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

            arcpy.env.extent = geom.extent # Extent object of the geometry
            print("Extent set to match uc: {}".format(arcpy.env.extent))
            
            try:          

                if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                    arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                ## Count number of GUA Blocks
                cnt = arcpy.GetCount_management(uatlLyr)
                print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                ## Create and Select points from GUA borders used to create SA
                print ("Create points from GUA borders used to create SA")

                #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                if arcpy.Exists(fc_GUA_pt_50m):
                    arcpy.Delete_management(fc_GUA_pt_50m)
                print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                nbrec = cnt.getOutput(0)
                print ("nb generated points : " + nbrec)

                if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                    arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                    arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                # save selection in a diferent feature class
                arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                # Adding FID field to the point feature class if it doesn't already exist
                if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                    arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                ids_ls_GQA = unique_values(uatlLyr, "FID")
                print("Total nb of GQA " +str(len(ids_ls_GQA)))

                # GQA without access to network (less than 25m)
                ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                
                # List to hold centroid points and their corresponding FIDs
                centroid_points = []
                fields_ls = ["FID", "SHAPE@", "area_m2"] # List of fields to include
                # Create a search cursor to iterate through the polygon shapefile GQA
                with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                    for row in cursor:
                        if row[0] in ids_noaccess:
                            # Get the FID and the geometry of the polygon
                            fid = row[0]
                            polygon = row[1]
                            area = row[2]
                            # Compute the centroid of the polygon
                            centroid = polygon.centroid
                            # Append the FID and centroid to the list
                            centroid_points.append((fid, centroid, area))

                # Start an edit session for the geodatabase
                with arcpy.da.Editor(outGDB) as edit:
                    # Create an insert cursor for the point feature class
                    with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                        for fid, point, area in centroid_points:
                            # Insert the centroid point and the FID
                            cursor.insertRow([fid, point, area])
                print("Centroids and FIDs computed and appended successfully.")

                # Add FID value to all records
                field_to_check = 'FID'
                alternative_field = 'ORIG_FID'
                #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                expression ="!FID! if !FID! is not None else !ORIG_FID!"
                # Execute CalculateField
                arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                nbrec = cnt.getOutput(0)
                print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
        
                if arcpy.Exists("{}GUA_points_layer".format(uc_code)):
                    arcpy.Delete_management("{}GUA_points_layer".format(uc_code))
                GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code))
                print("{}GUA_points_layer".format(uc_code))
                

                # Create service area layer
                ###create_service_area_layer(GUA_pt_lyr, "{}_SA_400m".format(uc_code), pedestrianNWpath, uc_code, nbMeters)
                group_id = uc_code
                network_dataset = pedestrianNWpath
                locations = GUA_pt_lyr
                create_service_area_layer(group_id, network_dataset, locations, fc_SA)

                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                
            except CustomError as ce:
                arcpy.AddError(ce.value)
                print (ce.value)
            except arcpy.ExecuteError:
                msgs = arcpy.GetMessages(2)
                arcpy.AddError(msgs)
                print (msgs)
            except:
                tb = sys.exc_info()[2]
                tbinfo = traceback.format_tb(tb)[0]
                pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                arcpy.AddError(pymsg)
                print (msgs)
                arcpy.AddError(msgs)
                print (pymsg)


Network Analyst extension has been checked out.
GEOSTAT21_901
Helsinki
start: 15:51:20
Extent set to match uc: 5129000 4201000 5157000 4228000 NaN NaN NaN NaN
nb record(s) in GEOSTAT21_901_finalGQA.shp subset selection [code_2018] IN (14100,30000,31000) : 3748
Create points from GUA borders used to create SA
GeneratePointsAlongLines Distance 50m from selected GUA in GEOSTAT21_901_finalGQA.shp to GEOSTAT21_901_GUA_pt_50m
nb generated points : 61066
SelectLayerByLocation points in GEOSTAT21_901_GUA_pt_50m WITHIN_A_DISTANCE of 25 meters from nw to GEOSTAT21_901_GUA_pt_50m_nw_25m
Nb of GQA near network 3356
Total nb of GQA 3748
Centroids and FIDs computed and appended successfully.
nb points in GEOSTAT21_901_GUA_pt_50m_nw_25m : 37878
GEOSTAT21_901GUA_points_layer
Creating network layer for batch GEOSTAT21_901
Solving network analysis
end: 16:55:32
GEOSTAT21_925
Wuppertal
GEOSTAT21_334
Solingen
GEOSTAT21_363
Bonn
GEOSTAT21_297
Moers / Rheinhausen
GEOSTAT21_405
Wiesbaden
GEOSTAT21_408
Mainz


In [2]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(group_id, network_dataset, locations, fc_SA):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source= network_dataset,
        layer_name="{}_SA_400m".format(group_id),
        travel_mode="Walk",
        travel_direction="FROM_FACILITIES",
        cutoffs=[400],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS",
        polygon_detail="HIGH",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="10 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )
    arcpy.na.AddLocations(
        in_network_analysis_layer=salyr,
        sub_layer="Facilities",        
        in_table=locations,
        field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Meters # 0;Attr_pedestrian_minutes # 0;Breaks_Minutes # #;Breaks_Meters # #;Breaks_pedestrian_minutes # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="nw SHAPE;Network_ND_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="NO_SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )
    print("Solving network analysis")
    arcpy.na.Solve(
        in_network_analysis_layer=salyr,
        ignore_invalids="SKIP",
        terminate_on_solve_error="CONTINUE",
        simplification_tolerance=None,
        overrides=""
    )
    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
    arcpy.Delete_management(salyr) 

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'step1_GQA')
step2_f = os.path.join(indata_f, 'OutputData', 'step2_GQA_Final')
step3_f = os.path.join(indata_f, 'OutputData', 'step3_SA')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_Input_MultiUA.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"

    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for ctry
ctry = 'DE'
sql_query = "Batch >0 "

# Define the path and name for the geodatabase
gdb_path = os.path.join(indata_f, 'Processing')

gdb_name = f"SA_output_1911_{ctry}.gdb"
outGDB = os.path.join(gdb_path, gdb_name)
# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
        inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
        fc_SA = os.path.join(outPath, "{}_SA.shp".format(uc_code))
        ####fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
        if ctr_cd.startswith("CH-"):
            ctr_cd = "CH"

        ##if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO" and ctr_cd!="CH":
        if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd==ctry:
            print ("start: " + time.strftime("%H:%M:%S", time.localtime()))
            pedestrianNWpath = pedestrian_path + r"\NW_" + ctr_cd + ".gdb\\" + ctr_cd + r"\Network_ND"
            if not arcpy.Exists(pedestrianNWpath):
                raise CustomError("Featureclass " + pedestrianNWpath + " NOT FOUND")

            arcpy.env.extent = geom.extent # Extent object of the geometry
            print("Extent set to match uc: {}".format(arcpy.env.extent))
            
            try:          

                if arcpy.Exists("{}_UATLlayer".format(uc_code)):
                    arcpy.Delete_management("{}_UATLlayer".format(uc_code))
                uatlLyr = arcpy.MakeFeatureLayer_management(inGQA, "{}_UATLlayer".format(uc_code))

                ## Count number of GUA Blocks
                cnt = arcpy.GetCount_management(uatlLyr)
                print ("nb record(s) in " + os.path.basename(inGQA) + " subset selection [" + CODE_Fld + "] IN (14100,30000,31000) : " +  cnt.getOutput(0))

                ## Create and Select points from GUA borders used to create SA
                print ("Create points from GUA borders used to create SA")

                #fc_GUA_pt_50m = outPath + "\\GUA_pt_50m.shp" ## modified to output folder
                fc_GUA_pt_50m = outGDB +"\\{}_GUA_pt_50m".format(uc_code)

                if arcpy.Exists(fc_GUA_pt_50m):
                    arcpy.Delete_management(fc_GUA_pt_50m)
                print ("GeneratePointsAlongLines Distance 50m from selected GUA in " + os.path.basename(inGQA) + " to " + os.path.basename(fc_GUA_pt_50m))
                arcpy.GeneratePointsAlongLines_management(uatlLyr, fc_GUA_pt_50m, 'DISTANCE', Distance='50 meters', Include_End_Points='END_POINTS')

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m)
                nbrec = cnt.getOutput(0)
                print ("nb generated points : " + nbrec)

                if arcpy.Exists('{}_GUA_PT_50m_lyr'.format(uc_code)):
                    arcpy.Delete_management('{}_GUA_PT_50m_lyr'.format(uc_code))
                arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m, '{}_GUA_PT_50m_lyr'.format(uc_code)) 

                fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)    
                if arcpy.Exists(fc_GUA_pt_50m_nw_25m):
                    arcpy.Delete_management(fc_GUA_pt_50m_nw_25m)
                nw_Path = pedestrian_path + r"\NW_" + ctr_cd + ".gdb" + "\\" + ctr_cd + "\\nw"
                print ("SelectLayerByLocation points in " + os.path.basename(fc_GUA_pt_50m) + " WITHIN_A_DISTANCE of 25 meters from " + os.path.basename(nw_Path)+ " to " + os.path.basename(fc_GUA_pt_50m_nw_25m))
                Selection = arcpy.SelectLayerByLocation_management('{}_GUA_PT_50m_lyr'.format(uc_code), 'WITHIN_A_DISTANCE', nw_Path, "25 meters", "NEW_SELECTION")
                # save selection in a diferent feature class
                arcpy.CopyFeatures_management(Selection, fc_GUA_pt_50m_nw_25m) 
                # Adding FID field to the point feature class if it doesn't already exist
                if 'FID' not in [f.name for f in arcpy.ListFields(fc_GUA_pt_50m_nw_25m)]:
                    arcpy.AddField_management(fc_GUA_pt_50m_nw_25m, 'FID', 'LONG')

                UATL_Ids_GUAs_nw_25m = unique_values(fc_GUA_pt_50m_nw_25m, "ORIG_FID")
                print("Nb of GQA near network " + str(len(UATL_Ids_GUAs_nw_25m)))
                ids_ls_GQA = unique_values(uatlLyr, "FID")
                print("Total nb of GQA " +str(len(ids_ls_GQA)))

                # GQA without access to network (less than 25m)
                ids_noaccess = [item for item in ids_ls_GQA if item not in UATL_Ids_GUAs_nw_25m]
                
                # List to hold centroid points and their corresponding FIDs
                centroid_points = []
                fields_ls = ["FID", "SHAPE@", "area_m2"] # List of fields to include
                # Create a search cursor to iterate through the polygon shapefile GQA
                with arcpy.da.SearchCursor(uatlLyr, fields_ls) as cursor:
                    for row in cursor:
                        if row[0] in ids_noaccess:
                            # Get the FID and the geometry of the polygon
                            fid = row[0]
                            polygon = row[1]
                            area = row[2]
                            # Compute the centroid of the polygon
                            centroid = polygon.centroid
                            # Append the FID and centroid to the list
                            centroid_points.append((fid, centroid, area))

                # Start an edit session for the geodatabase
                with arcpy.da.Editor(outGDB) as edit:
                    # Create an insert cursor for the point feature class
                    with arcpy.da.InsertCursor(fc_GUA_pt_50m_nw_25m, fields_ls) as cursor:
                        for fid, point, area in centroid_points:
                            # Insert the centroid point and the FID
                            cursor.insertRow([fid, point, area])
                print("Centroids and FIDs computed and appended successfully.")

                # Add FID value to all records
                field_to_check = 'FID'
                alternative_field = 'ORIG_FID'
                #expression = f"!{field_to_check}! if !{field_to_check}! is not None else !{alternative_field}!"
                expression ="!FID! if !FID! is not None else !ORIG_FID!"
                # Execute CalculateField
                arcpy.CalculateField_management(fc_GUA_pt_50m_nw_25m, field_to_check, expression, "PYTHON3")

                cnt = arcpy.GetCount_management(fc_GUA_pt_50m_nw_25m)
                nbrec = cnt.getOutput(0)
                print ("nb points in " + os.path.basename(fc_GUA_pt_50m_nw_25m) + " : " + nbrec)
        
                if arcpy.Exists("{}GUA_points_layer".format(uc_code)):
                    arcpy.Delete_management("{}GUA_points_layer".format(uc_code))
                GUA_pt_lyr = arcpy.MakeFeatureLayer_management(fc_GUA_pt_50m_nw_25m, "{}GUA_points_layer".format(uc_code))
                print("{}GUA_points_layer".format(uc_code))
                

                # Create service area layer
                ###create_service_area_layer(GUA_pt_lyr, "{}_SA_400m".format(uc_code), pedestrianNWpath, uc_code, nbMeters)
                group_id = uc_code
                network_dataset = pedestrianNWpath
                locations = GUA_pt_lyr
                create_service_area_layer(group_id, network_dataset, locations, fc_SA)

                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))
                
            except CustomError as ce:
                arcpy.AddError(ce.value)
                print (ce.value)
            except arcpy.ExecuteError:
                msgs = arcpy.GetMessages(2)
                arcpy.AddError(msgs)
                print (msgs)
            except:
                tb = sys.exc_info()[2]
                tbinfo = traceback.format_tb(tb)[0]
                pymsg = "PYTHON ERRORS:\nTraceback info:\n" + tbinfo + "\nError Info:\n" + str(sys.exc_info()[1])
                msgs = "ArcPy ERRORS:\n" + arcpy.GetMessages(2) + "\n"
                arcpy.AddError(pymsg)
                print (msgs)
                arcpy.AddError(msgs)
                print (pymsg)


Network Analyst extension has been checked out.
GEOSTAT21_901
Helsinki
GEOSTAT21_925
Wuppertal
GEOSTAT21_334
Solingen
start: 11:35:43
Extent set to match uc: 4105000 3113000 4120000 3125000 NaN NaN NaN NaN
nb record(s) in GEOSTAT21_334_finalGQA.shp subset selection [code_2018] IN (14100,30000,31000) : 202
Create points from GUA borders used to create SA
GeneratePointsAlongLines Distance 50m from selected GUA in GEOSTAT21_334_finalGQA.shp to GEOSTAT21_334_GUA_pt_50m
nb generated points : 3542
SelectLayerByLocation points in GEOSTAT21_334_GUA_pt_50m WITHIN_A_DISTANCE of 25 meters from nw to GEOSTAT21_334_GUA_pt_50m_nw_25m
Nb of GQA near network 171
Total nb of GQA 202
Centroids and FIDs computed and appended successfully.
nb points in GEOSTAT21_334_GUA_pt_50m_nw_25m : 1821
GEOSTAT21_334GUA_points_layer
Creating network layer for batch GEOSTAT21_334
Solving network analysis
end: 11:42:33
GEOSTAT21_363
Bonn
GEOSTAT21_297
Moers / Rheinhausen
GEOSTAT21_405
Wiesbaden
GEOSTAT21_408
Mainz
GEOST

In [9]:
#----------------------------------------------------------------------------------
#Author: Veerle Martens
#Date: may 2012
#Adapted by: Raquel Ubach (UAB) - Dec 2023/Jul 2024
#
#Purpose: Create service areas around each building block and find out the area of
#         green urban area that can be reached.
#---------------------------------------------------------------------------------

#create error handler
class CustomError(Exception):
    def __init__(self,value):
        self.value = value
    def __str__(self):
        return repr(self.value)

import sys, arcpy, time, traceback, os.path
import pandas as pd

def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

def convert(list):
    return tuple(list)

def create_service_area_layer(group_id, network_dataset, locations, fc_SA):
    print ("Creating network layer for batch {}".format(group_id))
    salyr = arcpy.na.MakeServiceAreaAnalysisLayer(
        network_data_source= network_dataset,
        layer_name="{}_SA_400m".format(group_id),
        travel_mode="Walk",
        travel_direction="FROM_FACILITIES",
        cutoffs=[400],
        time_of_day=None,
        time_zone="LOCAL_TIME_AT_LOCATIONS",
        output_type="POLYGONS",
        polygon_detail="HIGH",
        geometry_at_overlaps="OVERLAP",
        geometry_at_cutoffs="DISKS",
        polygon_trim_distance="10 Meters",
        exclude_sources_from_polygon_generation=None,
        accumulate_attributes=None,
        ignore_invalid_locations="SKIP"
    )
    arcpy.na.AddLocations(
        in_network_analysis_layer=salyr,
        sub_layer="Facilities",        
        in_table=locations,
        field_mappings="Name # #;CurbApproach # 0;Attr_Minutes # 0;Attr_Meters # 0;Attr_pedestrian_minutes # 0;Breaks_Minutes # #;Breaks_Meters # #;Breaks_pedestrian_minutes # #",
        search_tolerance="5000 Meters",
        sort_field=None,
        search_criteria="nw SHAPE;Network_ND_Junctions NONE",
        match_type="MATCH_TO_CLOSEST",
        append="APPEND",
        snap_to_position_along_network="NO_SNAP",
        snap_offset="5 Meters",
        exclude_restricted_elements="EXCLUDE",
        search_query=None,
        allow_auto_relocate="ALLOW"
    )
    print("Solving network analysis")
    arcpy.na.Solve(
        in_network_analysis_layer=salyr,
        ignore_invalids="SKIP",
        terminate_on_solve_error="CONTINUE",
        simplification_tolerance=None,
        overrides=""
    )
    arcpy.CopyFeatures_management("{}_SA_400m\Polygons".format(uc_code), fc_SA)
    arcpy.Delete_management(salyr) 

# Path to data folders
indata_f = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas'
networks_f = os.path.join(indata_f, 'Network data Regio')
step1_f = os.path.join(indata_f, 'OutputData', 'step1_GQA')
step2_f = os.path.join(indata_f, 'OutputData', 'step2_GQA_Final')
step3_f = os.path.join(indata_f, 'OutputData', 'step3_SA')
if not os.path.exists(step3_f):
    os.makedirs(step3_f)
uc_file_path = os.path.join(indata_f, 'UrbanCentres', 'HDC2021_RG_InputUpdateB2B3B4Copy.shp')

# input parameters
pedestrian_path = networks_f
outPath = step3_f 
CODE_Fld = "code_2018"
UATL_ID_Fld = "FID"

    
# TRANSLATOR TABLE
# Crosswalk table containing the different codes from input sources
codes_path = r'P:\Environment and Health\Noise\ServiceContract\2024_ServiceContract\QuietAreas\Processing\Codes.csv'
codes = pd.read_csv(codes_path)

#Environment settings
arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

try:
    arcpy.CheckOutExtension("Network")
    print("Network Analyst extension has been checked out.")
except arcpy.ExecuteError:
    print("Failed to check out the Network Analyst extension.")
    
# Run process for ctry
ctry = 'FI'
sql_query = "Batch >0"

# Define the path and name for the geodatabase
gdb_path = os.path.join(indata_f, 'Processing')

gdb_name = f"SA_output_1911_{ctry}.gdb"
outGDB = os.path.join(gdb_path, gdb_name)
# Check if the geodatabase exists
if not arcpy.Exists(outGDB):
    # Create the geodatabase if it does not exist
    arcpy.CreateFileGDB_management(gdb_path, gdb_name)

with arcpy.da.SearchCursor(uc_file_path, ["SHAPE@", "HDENS_CLST", "CNTR_CODE"], where_clause=sql_query) as uc_cursor:
    for uc_row in uc_cursor:
        geom = uc_row[0]  # Geometry object
        ctr_cd = uc_row[2]
        uc_code = uc_row[1]
        print(uc_code)
        print(codes.query(f'HDENS_CLST == "{uc_code}"').HDENS_NAME.values.astype(str)[0].strip())
        inGQA = os.path.join(step2_f, '{}_finalGQA.shp'.format(uc_code))
        fc_SA = os.path.join(outPath, "{}_SA.shp".format(uc_code))
        ####fc_SA = os.path.join(gdb_path, gdb_name, "{}_SA".format(uc_code))
        if ctr_cd.startswith("CH-"):
            ctr_cd = "CH"
        
        outDiss = os.path.join(outPath, 'Dissolved', "{}_SAd.shp".format(uc_code))
        fc_GUA_pt_50m_nw_25m = outGDB +"\\{}_GUA_pt_50m_nw_25m".format(uc_code)
        ##if os.path.exists(inGQA) and not arcpy.Exists(fc_SA) and ctr_cd!="NO" and ctr_cd!="CH":
        if os.path.exists(inGQA) and not arcpy.Exists(outDiss) and ctr_cd==ctry:
            print(fc_GUA_pt_50m_nw_25m)
            print(inGQA)
            print(outDiss)

            #join GQA FID
            arcpy.management.JoinField(
                in_data=fc_SA,
                in_field="FacilityID",
                join_table=fc_GUA_pt_50m_nw_25m,
                join_field="OBJECTID",
                fields="FID",
                fm_option="NOT_USE_FM",
                field_mapping=None,
                index_join_fields="NO_INDEXES"
            )
            # Dissolve by QGA id
            outDiss = os.path.join(outPath, 'Dissolved', "{}_SAd.shp".format(uc_code))
            if not os.path.exists(outDiss):
                arcpy.analysis.PairwiseDissolve(
                    in_features=fc_SA,
                    out_feature_class=outDiss,
                    dissolve_field="FID_1",
                    statistics_fields=None,
                    multi_part="MULTI_PART",
                    concatenation_separator=""
                )


                print ("end: " + time.strftime("%H:%M:%S", time.localtime()))







        print ("end: " + time.strftime("%H:%M:%S", time.localtime()))



Network Analyst extension has been checked out.
GEOSTAT21_018
Stockholm
end: 13:09:56
GEOSTAT21_021
Tartu
end: 13:09:56
GEOSTAT21_022
Örebro
end: 13:09:57
GEOSTAT21_023
Stavanger
end: 13:09:57
GEOSTAT21_024
Norrköping
end: 13:09:57
GEOSTAT21_025
Linköping
end: 13:09:57
GEOSTAT21_914
Jönköping / Huskvarna
end: 13:09:57
GEOSTAT21_029
Göteborg
end: 13:09:57
GEOSTAT21_026
Borås
end: 13:09:57
GEOSTAT21_028
Rīga
end: 13:09:57
GEOSTAT21_031
Aalborg
end: 13:09:57
GEOSTAT21_047
Århus
end: 13:09:58
GEOSTAT21_048
Helsingborg
end: 13:09:58
GEOSTAT21_903
København
end: 13:09:58
GEOSTAT21_049
Lund
end: 13:09:58
GEOSTAT21_051
Kaunas
end: 13:09:58
GEOSTAT21_054
Malmö
end: 13:09:58
GEOSTAT21_060
Odense
end: 13:09:58
GEOSTAT21_065
Gdynia
end: 13:09:58
GEOSTAT21_068
Gdańsk
end: 13:09:58
GEOSTAT21_904
Dublin
end: 13:09:58
GEOSTAT21_070
Elbląg
end: 13:09:59
GEOSTAT21_086
Kiel
end: 13:09:59
GEOSTAT21_090
Olsztyn
end: 13:09:59
GEOSTAT21_093
Lütten Klein
end: 13:09:59
GEOSTAT21_094
Rostock
end: 13:09:59
GEOST

In [8]:
fc_GUA_pt_50m_nw_25m

'P:\\Environment and Health\\Noise\\ServiceContract\\2024_ServiceContract\\QuietAreas\\Processing\\SA_output_1911_FI.gdb\\GEOSTAT21_867_GUA_pt_50m_nw_25m'